# Evaluation du modèle Mask R-CNN

Ce notebook complète l'article publié sur le blog de Makina Corpus à propos de l'**extraction d'objets pour la cartographie par deep-learning**, et plus particulièrement la partie concernant l'[évaluation du modèle](https://makina-corpus.com/blog/metier/2020/extraction-dobjets-pour-la-cartographie-par-deep-learning-evaluation-du-modele)

Ce notebook fait suite au notebook [Utilisation_de_Mask-RCNN](https://github.com/makinacorpus/tutorials/blob/master/deep_learning/notebooks/Utilisation_de_Mask-RCNN.ipynb) qui présente comment utiliser l'implémentation du modèle Mask R-CNN (disponible ici : [Matterport-Mask RCNN](https://github.com/matterport/Mask_RCNN/blob/master)). Il nécessite les mêmes installations :
* **`tensorflow`** et **`keras`**
* **`pycocotools`** (voir commande ci-dessous)
```console
pip install git+https://github.com/waleedka/coco.git#subdirectory=PythonAPI
 ```

### Imports 

In [1]:
#IMPORT LIBRAIRIES
import os
import sys
import datetime

#pycocotools
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

#mrcnn
from mrcnn import visualize
from mrcnn.config import Config
from mrcnn import model as modellib, utils


Using TensorFlow backend.


## Définition des variables et des dossiers nécessaires 

Nous utiliserons dans ce tutoriel les poids du modèle déjà entraîné : [submitted_weights.h5](https://gitlab.com/LaurentS/crowdai-mapping-challenge-mask-rcnn/-/blob/master/submitted_weights.h5) de cette [solution](https://gitlab.com/LaurentS/crowdai-mapping-challenge-mask-rcnn)
 au challenge Crowd AI Mapping challenge.
 
Le dossier `"logs"` correspond au dossier dans lequel les différents poids obtenus par l'entraînement du modèle seront enregistrés. 

In [2]:
#Preparation des chemins
ROOT_DIR = ""
PRETRAINED_MODEL_PATH ="submitted_weights.h5"
LOGS_DIRECTORY = os.path.join(ROOT_DIR, "logs")
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

## EVALUATION


### Initialisation du modèle en mode inférence 

Avec le fichier contenant les poids , il est possible de réaliser des évaluations. Pour cela nous devons configurer le modèle et l'intialiser en mode inférence.

#### Configuration du modèle pour l'évaluation

La configuration du modèle va varier selon le mode de fonctionnement du modèle. L'implémentation de Mask R-CNN contient déjà une classe pour la configuration du modèle : la classe `Config` du fichier `mrcnn.config`.

Il suffit donc de créer une classe qui hérite de celle-ci pour configurer le modèle à notre utilisation. 


In [14]:
### CONFIGURATION DU MODELE POUR LA PREDICTION ET L EVALUATION

class Buildings_Inference(Config):
    NAME = "Buildings_Detection"
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1  # Nous avons une classe "Buildings" , le background étant la classe par défaut , cela donne : 1 Background + 1 Building
    IMAGE_MAX_DIM=256
    IMAGE_MIN_DIM=256
    DETECTION_MIN_CONFIDENCE = 0.90 # c'est le seuil de confiance (score de prédiction) à partir du quel on veut que le modèle valide la détection de l'objet.

#initialisation de la configuration du modèle   
config = Buildings_Inference()

#visualisation de la configuration
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.999
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE           

#### Initialisation du modèle en mode inférence 

Nous initialisons le modèle en mode inférence avec la configuration définie précédemment.
Comme nous avons à notre disposition un modèle déjà entraîné , nous pouvons charger les poids de ce modèle pour la prédiction appliquée à nos données.

In [17]:
# TELECHARGEMENT DU MODELE ET CHARGEMENT DES POIDS DU MODELE PRE-ENTRAINE
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config) 
model_path = PRETRAINED_MODEL_PATH
model.load_weights(model_path, by_name=True) #téléchargement des poids du modèle déja entrainé
class_names = ['BG', 'buildings'] # Liste contenant le nom des catégories, permettant la visualisation du nom des catégories détectées

### Préparation du jeu de données

#### Configuration de la classe `BuildingDataset()`

Pour se préparer à l'évaluation du modèle, il faut permettre au modèle de parcourir notre jeu de vérité terrain : pour cela, nous utilisons la classe `BuildingDataset()`.
Elle hérite de la classe `Dataset()` de `mrcnn.utils` et s'adapte à notre vérité terrain.

Sa principale fonctionnalité est de charger le fichier d'annotations et d'en extraire les informations qui y sont contenues ainsi que de permettre la conversion des masques dans un format pouvant être lu et utilisé par le modèle.

In [ ]:
#CLASSE PERMETTANT LA CONFIGURATION DU DATASET

class BuildingDataset(utils.Dataset):
    
    """ Part 1 : Download Annotation file """
    
    def load_dataset(self,annotation_file,image_dir,return_coco=True, limit=None):
        """
        return COCO dataset
        """
        # COCO DATASET
        self.coco = COCO(annotation_file)
        self.image_dir =image_dir
        
        # DATASET INFORMATIONS
        # Load all classes (Only Building in this version)
        classIds = self.coco.getCatIds()
        # Load all images
        image_ids = list(self.coco.imgs.keys())[:limit]
        print(len(image_ids), 'images loaded')
        # register classes
        for _class_id in classIds:
            self.add_class("buildings_data", _class_id, self.coco.loadCats(_class_id)[0]["name"])
        # Register Images
        for _img_id in image_ids:
            assert(os.path.exists(os.path.join(image_dir, self.coco.imgs[_img_id]['file_name'])))
            self.add_image(
                "buildings_data", image_id=_img_id,
                path=os.path.join(image_dir, self.coco.imgs[_img_id]['file_name']),
                width=self.coco.imgs[_img_id]["width"],
                height=self.coco.imgs[_img_id]["height"],
                annotations=self.coco.loadAnns(self.coco.getAnnIds(
                                            imgIds=[_img_id],
                                            catIds=classIds,
                                            iscrowd=0)))

        if return_coco:
            return self.coco
        
        
    """ Part : Download Mask """    

    def load_mask(self, image_id):
        """ Loads instance mask for a given image
              This function converts mask from the coco format to a
              a bitmap [height, width, instance]
            Params:
                - image_id : reference id for a given image

            Returns:
                masks : A bool array of shape [height, width, instances] with
                    one mask per instance
                class_ids : a 1D array of classIds of the corresponding instance masks)
        """

        image_info = self.image_info[image_id]
        assert image_info["source"] == "buildings_data"

        instance_masks = []
        class_ids = []
        annotations = self.image_info[image_id]["annotations"]
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.
        for annotation in annotations:
            class_id = self.map_source_class_id(
                "buildings_data.{}".format(annotation['category_id']))
            if class_id:
                m = self.annToMask(annotation,  image_info["height"],
                                                image_info["width"])
                # Some objects are so small that they're less than 1 pixel area
                # and end up rounded out. Skip those objects.
                if m.max() < 1:
                    continue

                # Ignore the notion of "is_crowd" as specified in the coco format
                # as we donot have the said annotation in the current version of the dataset

                instance_masks.append(m)
                class_ids.append(class_id)
        # Pack instance masks into an array
        if class_ids:
            mask = np.stack(instance_masks, axis=2)
            class_ids = np.array(class_ids, dtype=np.int32)
            
            
            return mask,class_ids  

        else:
            # return an empty mask if no instance for the given image.
            mask = np.empty([0, 0, 0])
            class_ids = np.empty([0], np.int32)
            return mask, class_ids

   
     
    """Part : Tools to convert masks """
    
    def annToRLE(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE to RLE.
        :return: binary mask (numpy 2D array)
        """
        segm = ann['segmentation']
        if isinstance(segm, list):
            # polygon -- a single object might consist of multiple parts
            # we merge all parts into one mask rle code
            rles = maskUtils.frPyObjects(segm, height, width)
            rle = maskUtils.merge(rles)
        elif isinstance(segm['counts'], list):
            # uncompressed RLE
            rle = maskUtils.frPyObjects(segm, height, width)
        else:
            # rle
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
        :return: binary mask (numpy 2D array)
        """
        rle = self.annToRLE(ann, height, width)
        m = maskUtils.decode(rle)
        return m

#### Initialisation du jeu de données pour l'évaluation

In [ ]:
#PREPARATION DU DATASET

# fchiers validation
annot_file_val="fichier annotation pour la validation.json"
img_directory_val="Dossier images pour la validation"

#validation dataset
data_val= BuildingDataset()
coco_data_val = data_val.load_dataset(annot_file_val,img_directory_val,return_coco=True)
data_val.prepare() 

### Fonctions permettant l'évaluation

L'évaluation va permettre de lancer un ensemble de prédictions et de comparer les résultats aux annotations de notre vérité terrain. Pour ce faire, nous utilisons les fonctions mises au point pour le dataset COCO.

Ces fonctions sont issues de https://github.com/matterport/Mask_RCNN/blob/master/samples/coco/coco.py

In [ ]:
# Fonction d'évaluation 

def evaluate_coco(model, dataset, coco, eval_type="bbox", limit=0, image_ids=None):
    """Runs official COCO evaluation.
    dataset: A Dataset object with vallidation data
    eval_type: "bbox" or "segm" for bounding box or segmentation evaluation
    limit: if not 0, it's the number of images to use for evaluation
    """
    # Pick COCO images from the dataset
    image_ids = image_ids or dataset.image_ids

    # Limit to a subset
    if limit:
        image_ids = image_ids[:limit]

    # Get corresponding COCO image IDs.
    coco_image_ids = [dataset.image_info[id]["id"] for id in image_ids]
    t_prediction = 0
    t_start = time.time()

    results = []

    for i, image_id in enumerate(image_ids):
        # Load image
        image = dataset.load_image(image_id)

        # Run detection
        t = time.time()
        print("="*100)
        print("Image shape ", image.shape)
        r = model.detect([image])
        r = r[0]
        t_prediction += (time.time() - t)
        print("Prediction time : ", (time.time() - t))
        # Convert results to COCO format
        image_results = build_coco_results(dataset, coco_image_ids[i:i + 1],
                                           r["rois"], r["class_ids"],
                                           r["scores"], r["masks"])
        print("Number of detections : ", len(r["rois"]))
        print("Classes Predicted : ", r["class_ids"])
        print("Scores : ", r["scores"])
        results.extend(image_results)

    # Load results. This modifies results with additional attributes.
    coco_results = coco.loadRes(results)

    # Evaluate
    cocoEval = COCOeval(coco, coco_results, eval_type)
    cocoEval.params.imgIds = coco_image_ids
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()
    

    print("Prediction time: {}. Average {}/image".format(
        t_prediction, t_prediction / len(image_ids)))
    print("Total time: ", time.time() - t_start)

In [ ]:
# Transformation des résultats de la prédiction du modèle en format COCO.
def build_coco_results(dataset, image_ids, rois, class_ids, scores, masks):
    """Arrange resutls to match COCO specs in http://cocodataset.org/#format
    """
    # If no results, return an empty list
    if rois is None:
        return []

    results = []
    for image_id in image_ids:
        # Loop through detections
        for i in range(rois.shape[0]):
            class_id = class_ids[i]
            score = scores[i]
            bbox = np.around(rois[i], 1)
            masks=masks.astype(np.uint8)
            mask = masks[:, :, i]
            y1, x1, y2, x2 = bbox
            result = {
                "image_id": image_id,
                "category_id": dataset.get_source_class_id(class_id, "buildings_data"),
                "bbox": [y1, x1, y2, x2 ],
                "score": score,
                "segmentation": maskUtils.encode(np.asfortranarray(mask))
            }
            results.append(result)
         
    return results

### Lancement de l'évaluation

Nous pouvons ainis lancer une évaluation du modèle sur notre jeu de données. L'option `eval_type` permet de définir ce que nous cherchons à évaluer. Dans cet exemple, l'option `"segm"` permet de lancer une évaluation sur la performance du modèle à prédire les masques associés aux bâtiments. Pour évaluer le modèle sur les boites englobantes prédites, il faut remplacer cette option par `"bbox"`.

In [ ]:
 # Evaluation du modèle avec 1000 images
evaluate_coco(model,dataset_val,val_coco, eval_type="segm",limit=1000) données

 Exemple d'un output obtenu suite à l'évalution :
 
 ```python
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.207
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.402
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.193
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.018
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.309
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.291
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.323
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.323
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.022
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.128
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.475
Prediction time: 2387.38641166687. Average 2.38738641166687/image
Total time:  2390.99524474144
```

**Le format de ce résultat d'évaluation est documenté dans l'[article en ligne sur le blog de Makina Corpus](https://makina-corpus.com/blog/metier/2020/extraction-dobjets-pour-la-cartographie-par-deep-learning-evaluation-du-modele).**